## Instalacja niezbędnych bibliotek

### Simple transformers

In [1]:
! pip install 'transformers==3.5.1'
! pip install 'simpletransformers==0.49.7'

In [2]:
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForMaskedLM, AutoModelWithLMHead, AutoModelForQuestionAnswering, AutoModelForPreTraining, AutoModelForCausalLM
import torch

herbert_model: AutoModel = AutoModelForMaskedLM.from_pretrained("allegro/herbert-base-cased")
herbert_tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
herbert_config: AutoConfig = AutoConfig.from_pretrained("allegro/herbert-base-cased")

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForMaskedLM: ['cls.sso.sso_relationship.weight', 'cls.sso.sso_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
dkleczek_tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1")
dkleczek_model: AutoModel = AutoModelForMaskedLM.from_pretrained("dkleczek/bert-base-polish-cased-v1")
dkleczek_config: AutoConfig = AutoConfig.from_pretrained("dkleczek/bert-base-polish-cased-v1")

Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
geo_tokenizer = AutoTokenizer.from_pretrained("Geotrend/bert-base-pl-cased")
geo_model = AutoModelForMaskedLM.from_pretrained("Geotrend/bert-base-pl-cased")
geo_config: AutoConfig = AutoConfig.from_pretrained("Geotrend/bert-base-pl-cased")


Wzorowane na przykładzie dr Smywiński-Pohl Aleksander: https://colab.research.google.com/drive/1XvUF4KfHSMlVafZRx7-YnDpuIAHULhLp?usp=sharing 

In [5]:
def predict_word(sequence, model, tokenizer):
    results_for_sentence = []
    sequence = sequence.replace("<mask>", tokenizer.mask_token)
    results_for_sentence.append(sequence)
    inputed = tokenizer.encode(sequence, return_tensors="pt")
    mask_token_index = torch.where(inputed == tokenizer.mask_token_id)[1][0].item()
    token_logits = model(inputed)
    mask_token_logits = token_logits[0][0, mask_token_index, :]
    top_5_tokens = torch.topk(mask_token_logits, 5)[1]
    for token in top_5_tokens:
        result = sequence.replace(tokenizer.mask_token, tokenizer.decode([token]))
        results_for_sentence.append(result)
    return results_for_sentence

In [6]:
import pandas as pd

In [7]:
examples = [
            "Warszawa to największe <mask>.",
            "Te zabawki należą do <mask>.",
            "Policjant przygląda się <mask>.",
            "Na środku skrzyżowania widać <mask>.",
            "Właściciel samochodu widział złodzieja z <mask>.",
            "Prezydent z premierem rozmawiali wczoraj o <mask>.",
            "Witaj, drogi <mask>."
]

models = [(herbert_model, herbert_tokenizer, "herbert"), (dkleczek_model, dkleczek_tokenizer, "dkleczek"), (geo_model, geo_tokenizer, "geotrend")]

final_results = dict()

for model, tokenizer, model_name in models:
    results_for_model = []
    for example in examples:
        result = predict_word(example, model, tokenizer)
        results_for_model += result
    final_results[model_name] = results_for_model
pd.options.display.max_colwidth = 200
pd.DataFrame(data = {models[0][2]: final_results[models[0][2]],
                     models[1][2]: final_results[models[1][2]],
                     models[2][2]: final_results[models[2][2]],
                     })

,herbert,dkleczek,geotrend
0,Warszawa to największe <mask>.,Warszawa to największe [MASK].,Warszawa to największe [MASK].
1,Warszawa to największe miasto.,Warszawa to największe miasto.,Warszawa to największe miasto.
2,Warszawa to największe lotnisko.,Warszawa to największe województwo.,Warszawa to największe miasta.
3,Warszawa to największe centrum.,Warszawa to największe lotnisko.,Warszawa to największe woj.
4,Warszawa to największe miasta.,Warszawa to największe miasteczko.,Warszawa to największe Warszawa.
5,Warszawa to największe atrakcje.,Warszawa to największe państwo.,Warszawa to największe miast.
6,Te zabawki należą do <mask>.,Te zabawki należą do [MASK].,Te zabawki należą do [MASK].
7,Te zabawki należą do rodziny.,Te zabawki należą do ciebie.,Te zabawki należą do tzw.
8,Te zabawki należą do nas.,Te zabawki należą do mnie.,Te zabawki należą do pt.
9,Te zabawki należą do nich.,Te zabawki należą do nas.,Te zabawki należą do pl.


In [8]:

examples = [
  "Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie <mask>.",
  "Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie <mask>."
]


final_results = dict()

for model, tokenizer, model_name in models:
    results_for_model = []
    for example in examples:
        result = predict_word(example, model, tokenizer)
        results_for_model += result
    final_results[model_name] = results_for_model
pd.options.display.max_colwidth = 200
pd.DataFrame(data = {models[0][2]: final_results[models[0][2]],
                     models[1][2]: final_results[models[1][2]],
                     models[2][2]: final_results[models[2][2]],
                     })

,herbert,dkleczek,geotrend
0,"Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie <mask>.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie [MASK].","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie [MASK]."
1,"Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zdziwił.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zgodził.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie ##wiedział."
2,"Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie poddał.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie dowiedział.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie było."
3,"Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie dowiedział.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie martwił.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie stało."
4,"Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zastanawiał.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie bał.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie ##dził."
5,"Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie przyznał.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zabił.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie ##dzi."
6,"Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie <mask>.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie [MASK].","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie [MASK]."
7,"Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie dowiedziała.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie zgodziła.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie było."
8,"Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie przyznała.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie bała.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie ##wia."
9,"Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie bała.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie dowiedziała.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie stało."


In [9]:

examples = [
  "<mask> wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.",
  "W wakacje odwiedziłem <mask>, który jest stolicą Islandii.",
  "Informatyka na <mask> należy do najlepszych kierunków w Polsce.",
]


final_results = dict()

for model, tokenizer, model_name in models:
    results_for_model = []
    for example in examples:
        result = predict_word(example, model, tokenizer)
        results_for_model += result
    final_results[model_name] = results_for_model
pd.options.display.max_colwidth = 200
pd.DataFrame(data = {models[0][2]: final_results[models[0][2]],
                     models[1][2]: final_results[models[1][2]],
                     models[2][2]: final_results[models[2][2]],
                     })

,herbert,dkleczek,geotrend
0,"<mask> wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","[MASK] wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","[MASK] wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza."
1,"Woda wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Woda wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Jego wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza."
2,"Słońce wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Mięso wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Za wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza."
3,"Ziemia wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Słońce wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Po wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza."
4,"Następnie wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Nie wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Nie wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza."
5,"Ciało wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Piwo wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Ich wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza."
6,"W wakacje odwiedziłem <mask>, który jest stolicą Islandii.","W wakacje odwiedziłem [MASK], który jest stolicą Islandii.","W wakacje odwiedziłem [MASK], który jest stolicą Islandii."
7,"W wakacje odwiedziłem Kraków, który jest stolicą Islandii.","W wakacje odwiedziłem kraj, który jest stolicą Islandii.","W wakacje odwiedziłem Island, który jest stolicą Islandii."
8,"W wakacje odwiedziłem Oslo, który jest stolicą Islandii.","W wakacje odwiedziłem Cypr, który jest stolicą Islandii.","W wakacje odwiedziłem Reykjavík, który jest stolicą Islandii."
9,"W wakacje odwiedziłem Londyn, który jest stolicą Islandii.","W wakacje odwiedziłem Meksyk, który jest stolicą Islandii.","W wakacje odwiedziłem Porto, który jest stolicą Islandii."


### Which of the models produced the best results? / Który model wykazał się najlepszymi rezultatami?
Zależy w jakim zakresie. Ogólnie mówiąc najwięcej sensownych uzupełnień zdań zaproponował model allegro/herbert-base-cased.


### Was any of the models able to capture Polish grammar? / Czy któryś model poradził sobie z polską gramatyką?
Dwa pierwsze testowane modele (allegro/herbert-base-cased oraz dkleczek/bert-base-polish-cased-v1) radziły sobie z polską gramatyką całkiem dobrze, natomias model Geotrend/bert-base-pl-cased wydaje się dość często mylić. 

### Was any of the models able to capture long-distant relationships between the words? / Czy któryś model był w stanie poradzić sobie z dalekimi zależnościami pomiędzy słowami?
Pierwsze dwa modele są w stanie dostrzegać te zależności nawet w długich zdaniach. Ostatni model ma większe problemy.


### Was any of the models able to capture world knowledge? / Czy któryś model wykazał się znajomością świata?
Ostatni model Geotrend/bert-base-pl-cased - "zna się" na geografii, jako jedyny potrafił rozpoznać stolicę Islandii, natomiast w innych aspektach raczej nie wykazuje się wiedzą. 

Model allegro/herbert-base-cased radzi sobie nieźle, jeżeli chodzi o nazwy uczelni w Polsce, podał poprawną odpowiedź dla substancji wrzącej w 100 stopniach, jednak nie rozpoznał poprawnie stolicy Islandii. 

Model dkleczek/bert-base-polish-cased-v1 zdaje się mieć bardziej ogólną wiedzę niż model herberta i nie wskazał konkretnych uczelni. Nie odróżnia miast od państw, ale "wie", że woda wrze w 100 stopniach. 

### What are the most striking errors made by the models? / Jakie są najbardziej uderzające błędy modeli?
Informatyka na wsi należy do najlepszych kierunków w Polsce. -- nie ma za dobrego internetu na wsiach w Polsce, więc trudno się zgodzić z tym stwierdzeniem.

Nie/Mięso/Słońce/Ziemia wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza. - zdania błędne

Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zabił.	
Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie zabiła. - tego prawdopodobnie nikt nie mógłby powiedzieć, bo by nie żył

Na środku skrzyżowania widać rok/obraz/św. - zdania błędne

Warszawa to największe Warszawa. - zdanie błędne

Witaj, drogi ##erk. - i podobne zdania, które zwracał ostatni model

